This notebook performs a comprehensive becnhmark suit for the inference procedure for the double pendulum system using the RxInfer framework.

_Author: Dmitry Bagaev_

In [1]:
using DrWatson

In [2]:
@quickactivate "RxInferThesisExperiments"

In [28]:
using RxInferThesisExperiments, RxInfer, StaticArrays, Plots, PGFPlotsX, LaTeXStrings
using LinearAlgebra, StableRNGs, Random, BenchmarkTools, ColorSchemes, Dates, DataFrames

In [79]:
const bfolder = datadir("nlds", "rxinfer")

"/Users/bvdmitri/.julia/dev/thesis/data/nlds/rxinfer"

In [75]:
# Pregenerate paths for benchmark data
mkpath(bfolder);

In [5]:
# Create default environment with default parameters
const environment = DoublePendulum()

DoublePendulum()

In [6]:
# Define state-transition function, uses RK4 method internally, see the `src/` folder
f(state) = state_transition(environment)(state)

f (generic function with 1 method)

In [69]:
# Include the model specification
include(srcdir("models", "rxinfer", "doublependulum.jl"));

In [70]:
function run_benchmark(params)
    @unpack T, niterations, seed = params
    
    states, observations = rand(StableRNG(seed), environment, T);
    model    = double_pendulum(T)
    result   = run_inference(model, observations; iterations = niterations)
    e_states = extract_posteriors(T, result)
    amse     = compute_amse(states, e_states)
    
    benchmark_modelcreation = @benchmark RxInfer.create_model(double_pendulum($T), 
        meta = double_pendulum_meta(), 
        constraints = double_pendulum_constraints(),
    )
    
    benchmark_inference = @benchmark run_inference(model, observations; iterations = $niterations) setup=begin
        model = double_pendulum($T)
        states, observations = rand(StableRNG($seed), environment, $T);
    end
    
    emse = compute_emse(seed) do _seed
        local states, observations = rand(StableRNG(_seed), environment, T; random_start = true);
        local model    = double_pendulum(T)
        local result   = run_inference(model, observations; iterations = niterations)
        local e_states = extract_posteriors(T, result)
        return compute_amse(states, e_states)
    end
    
    output = @strdict T niterations seed states e_states observations amse emse benchmark_modelcreation benchmark_inference
    
    return output
end

run_benchmark (generic function with 1 method)

In [71]:
# Here we create a list of parameters we want to run our benchmarks with
benchmark_params = dict_list(Dict(
    "T"           => [ 100 ],
    "niterations" => [ 3, 5 ],
    "seed"        => [ 42 ]
));

In [85]:
# First run maybe slow, you may track the progress in the terminal
# Subsequent runs will not create new benchmarks 
# but will reload it from data folder
benchmarks = map(benchmark_params) do params
    path = datadir("nlds", "rxinfer")
    result, _ = produce_or_load(run_benchmark, bfolder, params; tag = false, force = true)
    return result
end;

[ Info: Producing file /Users/bvdmitri/.julia/dev/thesis/data/nlds/rxinfer/T=100_niterations=3_seed=42.jld2 now...
[ Info: File /Users/bvdmitri/.julia/dev/thesis/data/nlds/rxinfer/T=100_niterations=3_seed=42.jld2 saved.
[ Info: Producing file /Users/bvdmitri/.julia/dev/thesis/data/nlds/rxinfer/T=100_niterations=5_seed=42.jld2 now...
[ Info: File /Users/bvdmitri/.julia/dev/thesis/data/nlds/rxinfer/T=100_niterations=5_seed=42.jld2 saved.


In [86]:
prepare_benchmarks_table(bfolder)

[ Info: Scanning folder /Users/bvdmitri/.julia/dev/thesis/data/nlds/rxinfer for result files.
[ Info: Added 2 entries.


Row,T,seed,niterations,amse,emse,inference,creation
,Int64?,Int64?,Int64?,Float64?,Float64?,Tuple…?,Tuple…?
1,100,42,3,4.16954,1.28988,"(""15.0229ms"", ""21.913ms"", ""0.0ms"")","(""6.9531ms"", ""8.3678ms"", ""0.0ms"")"
2,100,42,5,4.15586,1.28286,"(""18.4389ms"", ""25.8704ms"", ""0.0ms"")","(""7.0223ms"", ""8.2064ms"", ""0.0ms"")"


# Versions

In [87]:
versioninfo()

Julia Version 1.9.0
Commit 8e630552924 (2023-05-07 11:25 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin22.4.0)
  CPU: 12 × Intel(R) Core(TM) i7-8850H CPU @ 2.60GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-14.0.6 (ORCJIT, skylake)
  Threads: 2 on 12 virtual cores


In [88]:
] status

Project RxInferThesisExperiments v1.0.0
Status `~/.julia/dev/thesis/Project.toml`
  [6e4b80f9] BenchmarkTools v1.3.2
  [35d6a980] ColorSchemes v3.21.0
  [a93c6f00] DataFrames v1.5.0
⌃ [31c24e10] Distributions v0.25.94
  [634d3b9d] DrWatson v2.12.5
  [9fc3f58a] ForneyLab v0.12.0
  [f6369f11] ForwardDiff v0.10.35
  [7073ff75] IJulia v1.24.0
  [b964fa9f] LaTeXStrings v1.3.0
  [3bd65402] Optimisers v0.2.18
  [8314cec4] PGFPlotsX v1.6.0
  [e4faabce] PProf v2.2.2
⌃ [91a5bcdd] Plots v1.38.12
  [37e2e3b7] ReverseDiff v1.14.6
⌃ [86711068] RxInfer v2.10.4
  [860ef19b] StableRNGs v1.0.0
  [aedffcd0] Static v0.8.7
  [90137ffa] StaticArrays v1.5.25
  [fce5fe82] Turing v0.25.1
  [37e2e46d] LinearAlgebra
  [9a3f8284] Random
Info Packages marked with ⌃ have new versions available and may be upgradable.
